Name              : 호칭만 원핫 인코딩 (5개 인코딩)

Sex               : 원핫 인코딩

Ticket            : 삭제

Cabin             : 삭제

Embarked          : 원핫 인코딩 - 생존율의 차이는 크게 없어서 추후에 제거 가능


### 수치형 데이터
Pclass        -0.338481  : 원핫 인코딩 해서 사용

Age           -0.077221  : 10살 범주로 나이대를 그룹화 한 후 사용(너무 적은 나이대의 범주데이터는 없애기) or 곧바로 표준화후 사용

SibSp         -0.035322  : 삭제

PassengerId   -0.005007  : 삭제

Parch          0.081629  : 삭제

Fare           0.257307  : 표준화 후 사용

In [185]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy

df_train_raw = pd.read_csv('train.csv')
df_test_raw = pd.read_csv('test.csv')
df_all_raw = pd.concat([df_train_raw, df_test_raw], axis=0)

df_train = copy.copy(df_train_raw)
df_test = copy.copy(df_test_raw)
df_all = copy.copy(df_all_raw)

df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [186]:
print(df_train.isnull().sum())
print()
print(df_test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [187]:
from sklearn.model_selection import train_test_split

In [188]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [189]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import copy


class myProcessor:
    
    def __init__(self, _df):
        self.df = copy.copy(_df)

    # 정상 작동 확인.
    def _nan_process(self, remove=True):
        # 결측치 처리 (옵션1, 결측치 행제거 / 옵션2, 결측치 평균값으로 메우기)
        if remove:
            self.df.dropna(inplace=True)
        else:
            # 평균값으로 결측치 메우기
            self.df.fillna(self.df.mean(), inplace=True)

    # 문제 해결.
    def _name_process(self):
        # Name : 호칭만 원핫 인코딩 (5개 인코딩)
        name_df = self.df['Name'].str.split(pat='[,.]', n=2, expand=True)
        self.df['honorific'] = name_df[1].str.strip()
        self.df['honorific'] = self.df['honorific'].apply(honorifics_classify)
        self.df.drop(columns=['Name'], inplace=True)

        # 원핫 인코딩
        df_honorific = self.df['honorific'].values
        honorific_ohe = OneHotEncoder()
        ohe_honorific = honorific_ohe.fit_transform(df_honorific.reshape(-1, 1)).toarray()
        honorific_list = honorific_ohe.categories_[0]
        df_ohe_honorific = pd.DataFrame(ohe_honorific, columns=honorific_list, index=self.df.index)

        # 축은 열 단위로 합쳐야 함 (axis=1)
        self.df = pd.concat([self.df, df_ohe_honorific], axis=1)
        self.df.drop(columns=['honorific'], inplace=True)

    # 문제없음
    def _remove(self):
        # Ticket, Cabin, PassengerId 삭제
        self.df.drop(columns=['Ticket', 'Cabin', 'PassengerId'], inplace=True)

    # 문제없다.
    #Parch 사용여부, Sibsp사용여부
    def _parch_sibsp_process(self, Parch_drop=True, SibSp_drop=True):
        # Parch 삭제 / 사용 선택
        if Parch_drop:
            self.df.drop(columns=['Parch'], inplace=True)

        # SibSp 삭제 / 사용 선택
        if SibSp_drop:
            self.df.drop(columns=['SibSp'], inplace=True)

    # 이새@끼가 문제일 가능성 100퍼다.
    def _data_ohe_encoding(self, embarked=True): # 옵션으로 embark제거 또는 활용 선택.
        # Sex : 원핫 인코딩
        df_sex = self.df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
        self.df['Sex'] = df_sex

        # Embarked : 원핫 인코딩
        if embarked:
            df_embarked = self.df['Embarked'].values
            embarked_ohe = OneHotEncoder()
            df_ohe_embarked = embarked_ohe.fit_transform(df_embarked.reshape(-1, 1)).toarray()
            embarked_list = embarked_ohe.categories_[0]
            df_ohe_embarked = pd.DataFrame(df_ohe_embarked, columns=embarked_list, index=self.df.index)
        self.df.drop(columns=['Embarked'], inplace=True)

        # Pclass : 원핫 인코딩
        df_pclass = self.df['Pclass'].values
        pclass_ohe = OneHotEncoder()
        df_ohe_pclass = pclass_ohe.fit_transform(df_pclass.reshape(-1, 1)).toarray()
        pclass_list = pclass_ohe.categories_[0]
        df_ohe_pclass = pd.DataFrame(df_ohe_pclass, columns=pclass_list, index=self.df.index)

        # Embarked가 True일 때만 df_ohe_embarked를 concat
        if embarked:
            self.df = pd.concat([self.df, df_ohe_embarked, df_ohe_pclass], axis=1)
        else:
            self.df = pd.concat([self.df, df_ohe_pclass], axis=1)


    def _age_process(self, used_age = False):
        if used_age:
            # 결측치의 행을 제거하고, Age를 10살 구간으로 나누고 그룹화
            bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
            labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
            self.df['AgeGroup'] = pd.cut(self.df['Age'], bins=bins, labels=labels)
            self.df.drop(columns=['Age'], inplace=True)
        else:
            self.df.drop(columns=['Age'], inplace=True)

    def _fare_process(self):
        # Fare : 표준화 후 사용
        self.df['Fare'] = (self.df['Fare'] - self.df['Fare'].mean()) / self.df['Fare'].std()


    '''
    # 결측치 처리 (옵션1, 결측치 행제거 / 옵션2, 결측치 평균값으로 메우기)
    # Parch 사용여부, Sibsp사용여부
    # 옵션으로 embark제거 또는 활용 선택.
    '''
    def preprocess_df(self, _remove=True, _Parch=False, _SibSp=False, _Embarked=True):
        self.remove = _remove
        self.Parch = _Parch
        self.SibSp = _SibSp
        self.Embarked = _Embarked

        self._remove()          #no
        self._nan_process(_remove) #ok
        self._age_process() #ok
        self._data_ohe_encoding(_Embarked) #ok
        self._name_process()  #no
        self._fare_process()  #ok 
        self._parch_sibsp_process(_Parch, _SibSp) #ok
        return self.df
    
    def get_splited_dataset(self, _test_size = 0.3, _ramdom_state = 0):
        X, y = self.df.drop(columns=['Survived']), self.df['Survived']
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size = _test_size,
                                                            random_state= _ramdom_state,
                                                            stratify=y)
        return X_train, X_test, y_train, y_test


def honorifics_classify(honorific):
    honorific_list = ['Mr', 'Miss', 'Mrs', 'Master']
    return honorific if honorific in honorific_list else 'else'


In [190]:
process1 = myProcessor(df_train)
preprocessed_df = process1.preprocess_df()
preprocessed_df.tail(5)

,Survived,Pclass,Sex,SibSp,Parch,Fare,C,Q,S,1,2,3,Master,Miss,Mr,Mrs,else
885,0,3,0,0,5,-0.102803,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
886,0,2,1,0,0,-0.407401,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1,1,0,0,0,-0.086274,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
889,1,1,1,0,0,-0.086274,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
890,0,3,1,0,0,-0.506572,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [191]:
X_train, X_test, y_train, y_test = process1.get_splited_dataset()
X_train

,Pclass,Sex,SibSp,Parch,Fare,C,Q,S,1,2,3,Master,Miss,Mr,Mrs,else
349,3,1,0,0,-0.489335,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
133,2,0,1,0,-0.161834,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
106,3,0,0,0,-0.508461,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
743,3,1,1,0,-0.348843,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
690,1,1,1,0,0.423750,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1,1,0,0,0.236741,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
585,1,0,0,2,0.851604,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
244,3,1,0,0,-0.516489,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
808,2,1,0,0,-0.407401,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
